In [1]:
%pip install linearmodels --quiet

import pandas as pd
import numpy as np
import io


import matplotlib.pyplot as plt

import statsmodels.formula.api as smf
from linearmodels import IV2SLS  # módulo específico de variables instrumentales

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.8 MB/s eta 0:00:00


# Introducción a la Inferencia Causal

# Ejercicios - Clase 3

## Ejercicio 1: Variables Instrumentales


Para este ejercicio usaremos los datos del trabajo de [Galiani, Rossi, & Schargrodsky (2011). Conscription and crime...](https://www.povertyactionlab.org/sites/default/files/research-paper/227_348%20Military%20conscription%20and%20crime%20in%20Argentina%20AER%20Apr%202011.pdf) que mencionamos durante la clase. Son datos reales.


Los autores quieren ver si ir al servicio militar en Argentina aumentaba las chances de luego cometer un crimen. Para poder identificar ese efecto aprovechan los sorteos anuales que se hacían para determinar quiénes debían cumplir con el servicio militar. Primero, se asignaba un número para cada grupo de personas de la cohorte correspondiente a ese año que tengan el DNI terminado en los mismos tres dígitos. Más tarde se anunciaba un número de corte y aquellos con número por encima debían hacer el servicio militar. Además, debían pasar un examen médico para poder hacerlo.

#### Datos

Para esta práctica usaremos los datos correspondientes a la cohorte de 1962.
Cada observación corresponde a un grupo de personas: aquellas que tienen los mismos 3 últimos números de DNI (por confidencialidad no se compartieron los DNI completos) en la cohorte de ese año.

Variables:
*   $sm_{i} $ es la proporción de personas con DNI terminado en $i$ que efectivamente hicieron el servicio militar. Es la variable de **tratamiento**, cuyo efecto sobre el crimen nos interesa estimar.
*   $numalto_{i} = 1$ si el número de la lotería para los DNIs terminados en $i$ en la cohorte $c$ fue "ALTO". Es una variable de **intención de tratamiento** (o _intention to treat_, ITT) . Los que fueron sorteados con número alto deberían ir al servicio militar, deberían recibir tratamiento.
*  $tasacrimen_{i} $ es la tasa de crimen para el grupo de personas con DNI terminado en $i$. Es la variable de **resultado**, sobre la que queremos medir el efecto.
*  $enfermedad_{i}$ es la proporción de personas del grupo $i$ que no pasan el examen médico.

In [2]:
# Importar datos
cc = pd.read_csv("https://storage.googleapis.com/humai-datasets/causalidad/consc_crime_1962.csv")
cc.head()

,sm,numalto,tasacrimen,enfermedad
0,0.020325,0,0.097978,0.089431
1,0.752101,1,0.040082,0.075630
2,0.689076,1,0.062349,0.100840
3,0.795349,1,0.111338,0.069767
4,0.682609,1,0.057896,0.095652


#### PARTE A: variables instrumentales para identificar el efecto causal

1. [TEÓRICA]. Si regresamos la tasa de crimen directamente en la variable $sm$ (participar o no en el servicio militar), ¿identificaríamos el efecto causal? ¿Por qué?


In [3]:
# Respuesta

print(
    '''
    No si hay algunas personas que voluntariamente deciden ir al servicio militar y otras que voluntariamente deciden evitarlo.
    Si es así, los grupos de tratamiento y control pueden tener características distintas que afecten al resultado (por ejemplo,
    ser más violentos), y no son comparables para estimar un efecto causal.
    ''')


    No si hay algunas personas que voluntariamente deciden ir al servicio militar y otras que voluntariamente deciden evitarlo. 
    Si es así, los grupos de tratamiento y control pueden tener características distintas que afecten al resultado (por ejemplo,
    ser más violentos), y no son comparables para estimar un efecto causal. 
    



2. [TEÓRICA]. Los autores usan $numalto$ para instrumentar $sm$. ¿Cuáles son las condiciones que deben cumplirse para poder identificar el efecto causal? ¿Cómo podemos mostrar que se cumplen?

In [4]:
# Respuesta

print(
    '''
    Relevancia y validez del intrumento.
    La relevancia regresando $sm$ en $numalto$ y analizando si esa correlación es significativa.
    La validez no es testeable, es el supuesto de identificación, debemos argumentar teóricamente que se cumple.
    ''')


    Relevancia y validez del intrumento.
    La relevancia regresando $sm$ en $numalto$ y analizando si esa correlación es significativa.
    La validez no es testeable, es el supuesto de identificación, debemos argumentar teóricamente que se cumple.
    



3. [PRÁCTICA]. Estimar la primera etapa de Variables Instrumentales. ¿$numalto$ es relevante para $sm$?


In [5]:
# Primera etapa
etapa1 = smf.ols("sm ~  1 + numalto", data = cc).fit()

print("estimador del parámetro de numalto ", etapa1.params["numalto"])
print("p-valor del parámetro de numalto: ", etapa1.pvalues["numalto"])


estimador del parámetro de numalto  0.685286843454905
p-valor del parámetro de numalto:  0.0


In [6]:
print("Sí, es relevante.")

Sí, es relevante.


4. [PRÁCTICA].  Estimar la segunda etapa. Estimar "de una sola vez" el estimador de variables instrumentales. ¿Hay efecto causal? ¿Qué sentido tiene?

In [7]:
# Segunda etapa
cc["smfit"] = etapa1.fittedvalues # valores predichos para sm
etapa2 = smf.ols("tasacrimen ~ 1 + smfit",
                     data = cc).fit()
etapa2.params["smfit"]

0.003745199275495881

In [8]:
# Estimamos "de una" usando linearmodels
formula = 'tasacrimen ~ 1 + [sm ~ numalto]'
iv = IV2SLS.from_formula(formula, data = cc).fit()

param = iv.params["sm"]
se = iv.std_errors["sm"]
p_val = iv.pvalues["sm"]

print(f"Parameter: {param}")
print(f"SE:  {se}")
print(f"95 CI: {(-1.96*se,1.96*se) + param}")
print(f"P-value: {p_val}")

Parameter: 0.0037451992754959497
SE:  0.0018127311639972014
95 CI: [0.00019225 0.00729815]
P-value: 0.038823443356233334


In [9]:
print('''
Hay un efecto causal significativo y positivo de asistir al servicio militar en el crimen.
Cada 1 punto que aumente la tasa de participación en el servicio militar, aumenta 1 punto
el crimen, para aquellos que van al servicio militar porque salieron sorteados.
''')


Hay un efecto causal significativo y positivo de asistir al servicio militar en el crimen.
Cada 1 punto que aumente la tasa de participación en el servicio militar, aumenta 1 punto
el crimen, para aquellos que van al servicio militar porque salieron sorteados.



#### PARTE B: cumplidores y no cumplidores


1.  [TEÓRICA]. ¿Qué significa en este ejemplo que estimamos el LATE y no el ATE?






In [10]:
print(
    '''
     Significa que es el efecto para los cumplidores, es decir, para aquellos cuya participación
     en el servicio militar depende exclusivamente de salir o no sorteados. Aquellos que de no
     haber salido sorteados no hubieran ido, pero que si salen sorteados sí van (ej. no mienten
     en el examen médico ni se anotan voluntariamente).
    '''
)


     Significa que es el efecto para los cumplidores, es decir, para aquellos cuya participación
     en el servicio militar depende exclusivamente de salir o no sorteados. Aquellos que de no 
     haber salido sorteados no hubieran ido, pero que si salen sorteados sí van (ej. no mienten 
     en el examen médico ni se anotan voluntariamente).
    


2.  [PRÁCTICA]. Estimar la diferencia de medias de cuántos aprueban el examen médico según si tienen o no número alto. Para hacerlo, se puede regresar $enfermedad$  en $numalto$. Los autores dicen que los resultados de esa diferencia de medias *sugieren que los exámenes médicos eran manipulados y que, por lo tanto, la participación en el servicio militar es endógena*. ¿Quiénes serían los *no-cumplidores* en este ejemplo?


In [11]:
trampa = smf.ols("enfermedad ~ 1 + numalto",
                     data = cc).fit()
trampa.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,0.0695,0.001,68.678,0.000,0.068,0.071
numalto,0.0232,0.001,18.946,0.000,0.021,0.026


In [12]:
print(
    '''
    Los no cumplidores son aquellos que a propósito buscan fallar en el examen médico.
    Los datos muestran que quienes salían sorteados tenían significativamente más chances de fallar en el examen,
    pero como la asignación es aleatoria es improbable que justo quienes tienen número alto tengan peor salud. Por eso
    los autores sostienen que es un indicio de que se hacía trampa en el examen.
    '''
)


    Los no cumplidores son aquellos que a propósito buscan fallar en el examen médico.
    Los datos muestran que quienes salían sorteados tenían significativamente más chances de fallar en el examen,
    pero como la asignación es aleatoria es improbable que justo quienes tienen número alto tengan peor salud. Por eso
    los autores sostienen que es un indicio de que se hacía trampa en el examen. 
    



3.  [PRÁCTICA] Estimar el ITT: regrese directamente la tasa de crimen en el número de sorteo. Grafique la relación entre tener número alto y la tasa de crimen. _Los autores estiman solamente el ITT para las cohortes 29-55 y 73-95, dado que no cuentan con el dato de tasa de participación en el servicio militar para esos años._

In [13]:
# Estimamos la forma reducida
reducida = smf.ols("tasacrimen ~ 1 + numalto", data=cc).fit()
print("estimación: ", reducida.params["numalto"])


estimación:  0.0025665357896142204


### Ejercicio 2: Diferencias en diferencias

Este es un pequeño ejercicio para usar el estimador de Diferencias en Diferencias.  Di Tella & Schagrodsky [en este trabajo](https://www.aeaweb.org/articles?id=10.1257/000282804322970733) quieren responder a la pregunta de si poner más policía en una cuadra ayuda a reducir el crimen.

Tras el atentado terrorista a la AMIA en julio de 1994, se decidió poner policía en aquellas cuadras de la Ciudad de Buenos Aires donde hubiera instituciones judías. Los autores aprovechan este aumento en la cantidad de policía para medir su efecto sobre el crimen. Cuentan con datos de robos de autos entre abril y diciembre de 1994 a nivel de cuadra, así como con la dirección donde se ubican las instituciones judías en varios barrios de la ciudad de Buenos Aires.

Noten que cuentan con datos _de panel_. El tratamiento (poner más policía) varía:
* en el tiempo: antes y después de julio de 1994
* en el espacio: entre cuadras con y sin instituciones


La unidad de observación, entonces, es una cuadra en un mes. Para cada cuadra y mes se cuenta con las siguientes variables:
* $totrob_cm$ es el número de robos de auto en la cuadra $c$ en el mes $m$. Si el robo sucede en una esquina, le asignan un 0.25 a cada cuadra.
* $instit_c = 1$ si hay una institución judía en esa cuadra ($0$ si no). Noten que esta variable sólo varía por cuadra, y es constante en el tiempo.
* $postjul_m = 1$ si el mes es posterior a julio de 1994, es decir, posterior al atentado. Noten que esta variable sólo varía en el tiempo, y es fija para todas las cuadras.


In [14]:
# Importamos los datos
amia = pd.read_csv("https://storage.googleapis.com/humai-datasets/causalidad/amia.csv")
amia.head()

,totrob,instit,postjul
0,0.0,0,0
1,0.0,0,0
2,0.0,1,0
3,0.0,0,0
4,0.0,1,0


1. Supongamos que sólo contamos con los datos de los meses posteriores al atentado. Calcular la diferencia en la cantidad de robos de autos entre las cuadras con y sin instituciones judías para esos meses.

In [15]:
tpost = amia.totrob[(amia.instit == 1) & (amia.postjul ==1)].mean()
cpost = amia.totrob[(amia.instit == 0) & (amia.postjul ==1)].mean()
print("Robos en cuadras con instituciones judías post atentado: " + str(tpost))
print("Robos en cuadras sin instituciones judías post atentado: " + str(cpost))
print("Diferencia: " + str(tpost - cpost))

Robos en cuadras con instituciones judías post atentado: 0.03513513513513514
Robos en cuadras sin instituciones judías post atentado: 0.10470798569725864
Diferencia: -0.0695728505621235


2. Supongamos ahora que contamos con datos para todos los meses pero solamente para las cuadras con instituciones judías. Calcular cómo varió la cantidad de robos antes y después del atentado.

In [16]:
tpre = amia.totrob[(amia.instit == 1) & (amia.postjul ==0)].mean()
print("Robos en cuadras con instituciones judías post atentado: " + str(tpost))
print("Robos en cuadras sin instituciones judías pre atentado: " + str(tpre))
print("Diferencia: " + str(tpost - tpre))

Robos en cuadras con instituciones judías post atentado: 0.03513513513513514
Robos en cuadras sin instituciones judías pre atentado: 0.11261261261261261
Diferencia: -0.07747747747747748


3. Ahora sí, contamos con todos los datos.

  i. Calcular cuál era la diferencia en la cantidad de robos de autos entre cuadras con instituciones judías y cuadras sin instituciones judías antes del atentado.

  ii. Calcular la variación en el número de robos de autos luego del atentado en las cuadras sin instituciones judías.

In [17]:
# diferencias previas controles y tratados, evolución controles
cpre = amia.totrob[(amia.instit == 0) & (amia.postjul ==0)].mean()

print(tpre - cpre)
print(cpost - cpre)

0.01745965273974809
0.009555025824394117


In [18]:
cpre = amia.totrob[(amia.instit == 0) & (amia.postjul ==0)].mean()

print("Diferencia en diferencia (dif. después - dif. antes)): " + str(  (tpost - cpost) - (tpre - cpre)  ))
print("Diferencia en diferencia (dif. tratados - dif. controles)): " + str(  (tpost - tpre) - (cpost-  cpre)  ))

Diferencia en diferencia (dif. después - dif. antes)): -0.0870325033018716
Diferencia en diferencia (dif. tratados - dif. controles)): -0.0870325033018716


4. Calcular el efecto de asignar más policía sobre la cantidad de robos de autos usando el método de diferencias en diferencias. ¿Cuál es el efecto de poner más policía en los niveles de crimen? Estimar diferencias en diferencias como una regresión lineal que incluye variables binarias. ¿El efecto es significativamente distinto de cero?

In [19]:
smf.ols('totrob ~ instit + postjul + instit*postjul', data=amia).fit().summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,0.0952,0.005,19.119,0.000,0.085,0.105
instit,0.0175,0.024,0.721,0.471,-0.030,0.065
postjul,0.0096,0.006,1.518,0.129,-0.003,0.022
instit:postjul,-0.0870,0.031,-2.841,0.005,-0.147,-0.027
